In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import resnet50
from torchvision.models.vgg import vgg16_bn
from torchvision.models.densenet import densenet121
from torchvision.models.mobilenet import mobilenet_v2
from efficientnet_pytorch import EfficientNet
from transformers import ViTModel, ViTFeatureExtractor
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from learn2learn.data import MetaDataset
import random
from PIL import Image
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch import nn


class PatchEmbedding(nn.Module):
    def __init__(self, patch_size=16, embed_dim=128):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.embedding_network = nn.Sequential(
            nn.Linear(patch_size * patch_size * 3, 256),
            nn.ReLU(),
            nn.Linear(256, embed_dim),
            nn.LayerNorm(embed_dim)
        )
        
    def forward(self, x):
        B, C, H, W = x.shape
        # Split image into patches
        patches = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        patches = patches.permute(0, 2, 3, 1, 4, 5)  # (B, H/16, W/16, C, patch_size, patch_size)
        
        # Flatten patches while keeping spatial information
        patch_vectors = patches.reshape(B, H//self.patch_size, W//self.patch_size, -1)
        
        # Create embeddings for each patch
        embeddings = self.embedding_network(patch_vectors)  # (B, 14, 14, embed_dim)
        
        return embeddings

class EfficientNetPatchB4(nn.Module):
    def __init__(self, patch_size=16, embed_dim=128):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b4')
        self.patch_embed = PatchEmbedding(patch_size, embed_dim)
        
    def forward(self, x):
        # Generate patch embeddings with spatial information preserved
        patch_embeddings = self.patch_embed(x)  # (batch_size, 14, 14, embed_dim)
        patch_embeddings = F.normalize(patch_embeddings, p=2, dim=-1)
        return patch_embeddings

class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super().__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2, x3):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        output3 = self.embedding_net(x3)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

class SpatialTripletLoss(nn.Module):
    def __init__(self, margin):
        super().__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = (anchor - positive).pow(2).sum(dim=-1)  # Sum over embedding dimension
        distance_negative = (anchor - negative).pow(2).sum(dim=-1)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

class TripletDataset(Dataset):
    def __init__(self, root=None, transforms=None):
        img_folder = ImageFolder(root=root)
        meta_data = MetaDataset(img_folder)
        
        self.img_list = img_folder.imgs
        self.labels_to_indices = meta_data.labels_to_indices
        self.indices_to_labels = meta_data.indices_to_labels
        self.labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
        self.transforms = transforms

    def __getitem__(self, index):
        img_anchor = self.img_list[index][0]
        label_anchor = self.img_list[index][1]

        idx_positive = random.choice(self.labels_to_indices[label_anchor])            
        img_positive = self.img_list[idx_positive][0]
        
        aux_class = random.choice(list(set(self.labels) - set([label_anchor])))
        idx_negative = random.choice(self.labels_to_indices[aux_class])
        img_negative = self.img_list[idx_negative][0]

        img_anchor = Image.open(img_anchor).convert('RGB')
        img_positive = Image.open(img_positive).convert('RGB')
        img_negative = Image.open(img_negative).convert('RGB')

        if self.transforms is not None:
            img_anchor = self.transforms(img_anchor)
            img_positive = self.transforms(img_positive)
            img_negative = self.transforms(img_negative)

        return (img_anchor, img_positive, img_negative), []

    def __len__(self):
        return len(self.img_list)

def get_triplet_dataloader(root=None, batch_size=1, transforms=None):
    dataset = TripletDataset(root=root, transforms=transforms)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

def get_train_transforms():
    return T.Compose([
        T.Resize((224, 224)),
        T.RandomHorizontalFlip(0.5),
        T.RandomVerticalFlip(0.5),
        T.RandomApply([T.RandomRotation(10)], 0.25),
        T.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def get_val_transforms():
    return T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def fit(train_loader, val_loader, model, loss_fn, optimizer, scheduler, n_epochs, device, log_interval=100):
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        
        for batch_idx, (data, _) in enumerate(train_loader):
            img_a, img_p, img_n = data
            img_a, img_p, img_n = img_a.to(device), img_p.to(device), img_n.to(device)
            
            optimizer.zero_grad()
            embeddings = model(img_a, img_p, img_n)
            loss = loss_fn(*embeddings)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            if batch_idx % log_interval == 0:
                print(f'Epoch {epoch}: [{batch_idx * len(img_a)}/{len(train_loader.dataset)}] '
                      f'Loss: {total_loss / (batch_idx + 1):.6f}')
        
        scheduler.step()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, _ in val_loader:
                img_a, img_p, img_n = data
                img_a, img_p, img_n = img_a.to(device), img_p.to(device), img_n.to(device)
                embeddings = model(img_a, img_p, img_n)
                val_loss += loss_fn(*embeddings).item()
        
        val_loss /= len(val_loader)
        print(f'Epoch {epoch}: Validation Loss: {val_loss:.6f}')

# Initialize model and training
path_data = 'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
embedding_net = EfficientNetPatchB4(patch_size=16, embed_dim=128)
triplet_model = TripletNet(embedding_net=embedding_net)
loss_fn = SpatialTripletLoss(margin=1.0)
optimizer = torch.optim.Adam(triplet_model.parameters(), lr=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    triplet_model = triplet_model.cuda()

# Create dataloaders
triplet_train_loader = get_triplet_dataloader(
    root=path_data + '/train/',
    batch_size=5,
    transforms=get_train_transforms()
)
triplet_val_loader = get_triplet_dataloader(
    root=path_data + '/val/',
    batch_size=5,
    transforms=get_val_transforms()
)

# Train
n_epochs = 100
fit(triplet_train_loader, triplet_val_loader, triplet_model, loss_fn, optimizer, scheduler, n_epochs, device)
torch.save(triplet_model, "f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/Patch_emdebbing_generator_triplet_model.h5" )

Loaded pretrained weights for efficientnet-b4
Epoch 0: [0/225] Loss: 0.842478
Epoch 0: Validation Loss: 0.928688
Epoch 1: [0/225] Loss: 0.998621
Epoch 1: Validation Loss: 0.929748
Epoch 2: [0/225] Loss: 1.019156
Epoch 2: Validation Loss: 0.922473
Epoch 3: [0/225] Loss: 0.990100
Epoch 3: Validation Loss: 0.927282
Epoch 4: [0/225] Loss: 0.846351
Epoch 4: Validation Loss: 0.922718
Epoch 5: [0/225] Loss: 0.890962
Epoch 5: Validation Loss: 0.910829
Epoch 6: [0/225] Loss: 0.899605
Epoch 6: Validation Loss: 0.909570
Epoch 7: [0/225] Loss: 0.859286
Epoch 7: Validation Loss: 0.910903
Epoch 8: [0/225] Loss: 0.922489
Epoch 8: Validation Loss: 0.902706
Epoch 9: [0/225] Loss: 0.900027
Epoch 9: Validation Loss: 0.904689
Epoch 10: [0/225] Loss: 0.905161
Epoch 10: Validation Loss: 0.902101
Epoch 11: [0/225] Loss: 0.946863
Epoch 11: Validation Loss: 0.895931
Epoch 12: [0/225] Loss: 1.061231
Epoch 12: Validation Loss: 0.900739
Epoch 13: [0/225] Loss: 0.892946
Epoch 13: Validation Loss: 0.900799
Epoch 14

In [5]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from datetime import datetime
from torch.utils.data import DataLoader
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
#sys.path.insert(0,'C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
sys.path.insert(0,'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
from dataloaders import get_val_transforms
import numpy as np
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms
from torch.autograd import Variable
import os
import pandas as pd
import seaborn as sns
import dataloaders
from dataloaders import get_train_transforms, get_val_transforms, get_triplet_dataloader
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from sklearn.metrics import accuracy_score, f1_score , precision_score , recall_score
from sklearn.manifold import TSNE
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from datetime import datetime

class RefinedViT(nn.Module):
    def __init__(self, num_classes=15):
        super(RefinedViT, self).__init__()
        self.num_patches = 196  # 14x14
        self.embed_dim = 128    # Match previous network
        self.num_heads = 8      
        
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.embed_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embed_dim,
            nhead=self.num_heads,
            dim_feedforward=512,
            dropout=0.1,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=10)
        
        self.norm = nn.LayerNorm(self.embed_dim)
        self.fc = nn.Linear(self.embed_dim, num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.reshape(batch_size, 196, -1)
        
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        
        x = self.transformer_encoder(x)
        x = x[:, 0]
        x = self.norm(x)
        x = self.dropout(x)
        return self.fc(x)

def generate_embeddings(data_loader, model, device):
    model.eval()
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        for batch_imgs, batch_labels in tqdm(data_loader):
            batch_imgs = batch_imgs.to(device)
            embeddings = model.get_embedding(batch_imgs)  # Using get_embedding from triplet model
            print(f"Size of embeddings: {embeddings.shape}")
            all_embeddings.append(embeddings.cpu().numpy())
            all_labels.append(batch_labels.numpy())
    
    return np.concatenate(all_embeddings), np.concatenate(all_labels)

def train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels, 
                      num_epochs=500, batch_size=32, learning_rate=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    X_train = torch.tensor(train_embeddings, dtype=torch.float32).to(device)
    y_train = torch.tensor(train_labels, dtype=torch.long).to(device)
    X_val = torch.tensor(val_embeddings, dtype=torch.float32).to(device)
    y_val = torch.tensor(val_labels, dtype=torch.long).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_train_loss = 0
        
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val).item()
            val_acc = (torch.argmax(val_outputs, dim=1) == y_val).float().mean().item()
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), 'best_model_refined_vit.pth')
        
        scheduler.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}: Train Loss={total_train_loss/len(X_train):.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')
    
    return model

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    path_data =  'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
    
    # Load triplet model
    #triplet_model = torch.load('C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/Patch_emdebbing_generator_triplet_model.h5', map_location=device)
    triplet_model.eval()
    
    # Data loaders
    transform = get_val_transforms()
    train_data = torchvision.datasets.ImageFolder(root=f'{path_data}/train/', transform=transform)
    val_data = torchvision.datasets.ImageFolder(root=f'{path_data}/val/', transform=transform)
    test_data = torchvision.datasets.ImageFolder(root=f'{path_data}/test/', transform=transform)
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    test_loader = DataLoader(test_data, batch_size=32)
    
    # Generate embeddings
    train_embeddings, train_labels = generate_embeddings(train_loader, triplet_model, device)
    val_embeddings, val_labels = generate_embeddings(val_loader, triplet_model, device)
    test_embeddings, test_labels = generate_embeddings(test_loader, triplet_model, device)
    
    # Train RefinedViT
    model = RefinedViT(num_classes=15)
    trained_model = train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels)
    
    return trained_model, (test_embeddings, test_labels)


model, test_data = main()  
torch.save(model, "f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/Patch_Rvit_triplet_model.h5" )

 38%|███████████████████████████████▌                                                    | 3/8 [00:00<00:00, 10.19it/s]

Size of embeddings: torch.Size([32, 14, 14, 128])
Size of embeddings: torch.Size([32, 14, 14, 128])
Size of embeddings: torch.Size([32, 14, 14, 128])


 75%|███████████████████████████████████████████████████████████████                     | 6/8 [00:00<00:00, 10.83it/s]

Size of embeddings: torch.Size([32, 14, 14, 128])
Size of embeddings: torch.Size([32, 14, 14, 128])
Size of embeddings: torch.Size([32, 14, 14, 128])


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 11.54it/s]


...

Epoch 480: Train Loss=0.0001, Val Loss=1.4053, Val Acc=0.6927


 98%|██████████████████████████████████████████████████████████████████████████████▍ | 490/500 [05:25<00:06,  1.52it/s]

Epoch 490: Train Loss=0.0002, Val Loss=1.4054, Val Acc=0.6927


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [05:32<00:00,  1.51it/s]

Epoch 500: Train Loss=0.0002, Val Loss=1.4054, Val Acc=0.6932


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW
from torchvision.datasets import ImageFolder

# Improved RefinedViT Model
class RefinedViT(nn.Module):
    def __init__(self, num_classes=15):
        super(RefinedViT, self).__init__()
        self.num_patches = 196  # 14x14
        self.embed_dim = 128    # Match previous network
        self.num_heads = 8      
        
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.embed_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embed_dim,
            nhead=self.num_heads,
            dim_feedforward=512,
            dropout=0.1,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=12)  # Increased layers
        
        self.norm = nn.LayerNorm(self.embed_dim)
        self.fc = nn.Sequential(
            nn.Linear(self.embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.reshape(batch_size, 196, -1)
        
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        
        x = self.transformer_encoder(x)
        x = x[:, 0]
        x = self.norm(x)
        x = self.dropout(x)
        return self.fc(x)

# Generate Embeddings
def generate_embeddings(data_loader, model, device):
    model.eval()
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        for batch_imgs, batch_labels in tqdm(data_loader):
            batch_imgs = batch_imgs.to(device)
            embeddings = model.get_embedding(batch_imgs)  # Using get_embedding from triplet model
            all_embeddings.append(embeddings.cpu().numpy())
            all_labels.append(batch_labels.numpy())
    
    return np.concatenate(all_embeddings), np.concatenate(all_labels)

# Train and Validate
def train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels, 
                      num_epochs=500, batch_size=32, learning_rate=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    X_train = torch.tensor(train_embeddings, dtype=torch.float32).to(device)
    y_train = torch.tensor(train_labels, dtype=torch.long).to(device)
    X_val = torch.tensor(val_embeddings, dtype=torch.float32).to(device)
    y_val = torch.tensor(val_labels, dtype=torch.long).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    best_val_loss = float('inf')
    early_stopping_patience = 10
    patience_counter = 0
    
    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_train_loss = 0
        
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            
            total_train_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val).item()
            val_acc = (torch.argmax(val_outputs, dim=1) == y_val).float().mean().item()
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), 'best_model_refined_vit.pth')
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= early_stopping_patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        
        scheduler.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}: Train Loss={total_train_loss/len(X_train):.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')
    
    return model

# Main Function
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    path_data = 'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
    
    # Load triplet model
    triplet_model = torch.load('f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/Patch_emdebbing_generator_triplet_model.h5', map_location=device)
    triplet_model.eval()
    
    # Data loaders
    transform = get_val_transforms()
    train_data = ImageFolder(root=f'{path_data}/train/', transform=transform)
    val_data = ImageFolder(root=f'{path_data}/val/', transform=transform)
    test_data = ImageFolder(root=f'{path_data}/test/', transform=transform)
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    test_loader = DataLoader(test_data, batch_size=32)
    
    # Generate embeddings
    train_embeddings, train_labels = generate_embeddings(train_loader, triplet_model, device)
    val_embeddings, val_labels = generate_embeddings(val_loader, triplet_model, device)
    test_embeddings, test_labels = generate_embeddings(test_loader, triplet_model, device)
    
    # Train RefinedViT
    model = RefinedViT(num_classes=15)
    trained_model = train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels)
    
    return trained_model, (test_embeddings, test_labels)

# Run the main function
model, test_data = main()  
torch.save(model, "f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/Patch_Rvit_triplet_model.h5")

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW
from torchvision.datasets import ImageFolder

# Improved RefinedViT Model
class RefinedViT(nn.Module):
    def __init__(self, num_classes=15):
        super(RefinedViT, self).__init__()
        self.num_patches = 196  # 14x14
        self.embed_dim = 128    # Match previous network
        self.num_heads = 16      
        
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.embed_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embed_dim,
            nhead=self.num_heads,
            dim_feedforward=512,
            dropout=0.1,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=12)  # Increased layers
        
        self.norm = nn.LayerNorm(self.embed_dim)
        self.fc = nn.Sequential(
            nn.Linear(self.embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.reshape(batch_size, 196, -1)
        
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        
        x = self.transformer_encoder(x)
        x = x[:, 0]
        x = self.norm(x)
        x = self.dropout(x)
        return self.fc(x)

# Generate Embeddings
def generate_embeddings(data_loader, model, device):
    model.eval()
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        for batch_imgs, batch_labels in tqdm(data_loader):
            batch_imgs = batch_imgs.to(device)
            embeddings = model.get_embedding(batch_imgs)  # Using get_embedding from triplet model
            all_embeddings.append(embeddings.cpu().numpy())
            all_labels.append(batch_labels.numpy())
    
    return np.concatenate(all_embeddings), np.concatenate(all_labels)

# Train and Validate
def train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels, 
                      num_epochs=500, batch_size=32, learning_rate=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    X_train = torch.tensor(train_embeddings, dtype=torch.float32).to(device)
    y_train = torch.tensor(train_labels, dtype=torch.long).to(device)
    X_val = torch.tensor(val_embeddings, dtype=torch.float32).to(device)
    y_val = torch.tensor(val_labels, dtype=torch.long).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    best_val_loss = float('inf')
    early_stopping_patience = 10
    patience_counter = 0
    
    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_train_loss = 0
        
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            
            total_train_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val).item()
            val_acc = (torch.argmax(val_outputs, dim=1) == y_val).float().mean().item()
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), 'best_model_refined_vit.pth')
                patience_counter = 0
            # else:
            #     patience_counter += 1
            #     if patience_counter >= early_stopping_patience:
            #         print(f"Early stopping at epoch {epoch+1}")
            #         break
        
        scheduler.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}: Train Loss={total_train_loss/len(X_train):.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')
    
    return model

# Main Function
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    path_data = 'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
    
    # Load triplet model
    triplet_model = torch.load('f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/Patch_emdebbing_generator_triplet_model.h5', map_location=device)
    triplet_model.eval()
    
    # Data loaders
    transform = get_val_transforms()
    train_data = ImageFolder(root=f'{path_data}/train/', transform=transform)
    val_data = ImageFolder(root=f'{path_data}/val/', transform=transform)
    test_data = ImageFolder(root=f'{path_data}/test/', transform=transform)
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    test_loader = DataLoader(test_data, batch_size=32)
    
    # Generate embeddings
    train_embeddings, train_labels = generate_embeddings(train_loader, triplet_model, device)
    val_embeddings, val_labels = generate_embeddings(val_loader, triplet_model, device)
    test_embeddings, test_labels = generate_embeddings(test_loader, triplet_model, device)
    
    # Train RefinedViT
    model = RefinedViT(num_classes=15)
    trained_model = train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels)
    
    return trained_model, (test_embeddings, test_labels)

# Run the main function
model, test_data = main()  
torch.save(model, "f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/Patch_Rvit_triplet_model.h5")

C:\Users\pc\AppData\Local\Temp\ipykernel_5148\3255058837.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  triplet_model = torch.load('f:/Meysam-Khodarahi/PlantDiseaseDi